In [26]:
from utils.config_loader import app_config
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore

from pymilvus import MilvusClient, DataType, db, connections, Collection, MilvusException, utility

from langchain_milvus import Milvus
from langchain_openai import OpenAIEmbeddings

import os

In [28]:
DATABASE_NAME = "default"

# define collection name
#COLLECTION_NAME = "KASEJKT032248"
COLLECTION_NAME = "_emails_attachments"

# host
HOST_IP = "172.20.140.34"
PORT = 19530

from utils.config_loader import app_config

host_ip = app_config.MILVUS_HOSTIP
port= app_config.MILVUS_PORT



In [29]:
# define the language model to be used for generating the vector embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    openai_api_key=os.getenv("OPENAI_API_KEY"),
)

In [30]:
from pymilvus import connections
from pymilvus import db

connection = connections.connect(
    host = host_ip ,
    port=port
)

db.using_database("default")

In [31]:
# Check the collection
from pymilvus import utility
collections = utility.list_collections()
print (collections)

['your_collection_name', '_emails_attachments']


In [ ]:
from pymilvus import Collection
collection = Collection("_emails_attachments")  # Get an existing collection.

collection.schema                # Return the schema.CollectionSchema of the collection.
collection.description           # Return the description of the collection.
collection.name                  # Return the name of the collection.
collection.is_empty              # Return the boolean value that indicates if the collection is empty.
collection.num_entities          # Return the number of entities in the collection.
collection.primary_field         # Return the schema.FieldSchema of the primary key field.
collection.partitions            # Return the list[Partition] object.
collection.indexes               # Return the list[Index] object.

In [33]:
collection = Collection("_emails_attachments")
print(collection.schema)

{'auto_id': False, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535}, 'is_primary': True, 'auto_id': False}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 1536}}], 'enable_dynamic_field': True}


In [ ]:
vector_db = Milvus(
   embedding_function = embeddings,
   collection_name = '_emails_attachments', 
   vector_field="embedding",  
)

In [35]:
retriever = vector_db.as_retriever(
    search_kwargs={"k":2}
)

In [36]:
system_prompt = \
"""
Use the following pieces of context to answer the question at the end.
if you don't know the answer based on the context only, say you do not
know the answer.

{context}
"""

In [37]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [39]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [40]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)

In [45]:
print(chain.input_schema())

ValidationError: 1 validation error for RunnableSequenceInput
root
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

In [42]:
response = chain.invoke({"input": "What is the quantity for fairy being shipped?"})
print(response)

KeyError: 'text'

### Example 1: Using LlamaIndex to query the vector store

In [ ]:
## Step 1: Prepare global settings

# Access appplication environment variables
model = app_config.LLAMAINDEX_MODEL
embed_model= app_config.LLAMAINDEX_EMBEDDING

# define global Setting that sets the LLM and ebemdding model.
# There is no need to pass in any llm or embedding model to the LlamaIndexTool query engine.
# The LlamaIndexTool will use the global Settings to get the llm and embedding model.
Settings.llm = OpenAI(temperature=0.1, model=model)
Settings.embed_model = OpenAIEmbedding(model=embed_model)

In [ ]:
# Step 2: Load index from existing Milvus vector store
index = VectorStoreIndex.from_vector_store(
    vector_store=milvus_store
)

In [ ]:
# Step 3. Create the query and chat engine using the LLM
query_engine = index.as_query_engine()  # no parameter needed, taken care off by the global Settings
chat_engine = index.as_chat_engine()


In [ ]:

# Step 4A. Run your query
response = query_engine.query("ecpls the contents of with shipment 2230500347?")
print(response)

In [ ]:
# Step 4B. Run your  cchat
response = chat_engine.chat("who is the owner of shipment 2230500347?")
print(response)